# 2. 搭建环境并实现简单的应用

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## 2.1 搭建环境

In [2]:
# curl -fsSL https://ollama.com/install.sh | sh
# ollama pull gpt-oss:20b
# venv/bin/pip3 install langchain==0.1.14 langchain-community==0.0.38 langchain-core==0.1.53 langchain-openai==0.1.7

import langchain
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
langchain.__version__

'0.1.14'

## 2.2 实现问答应用 — 基于基础模型

In [4]:
from langchain_community.llms import Ollama

llm = Ollama(model="gpt-oss:20b")
print(llm.predict("简要说明AI会对人类文明产生深远的影响吗？"))

**AI 对人类文明的深远影响（简要概述）**

| 领域 | 主要影响 | 典型案例/展望 |
|------|----------|--------------|
| **经济与劳动** | 生产效率翻倍、产业链重构；同时出现新的职业与旧职业的消失。 | 自动化工厂、AI 运营的金融交易、机器人客服。 |
| **社会结构** | 信息获取、消费习惯、社交方式大变；数字鸿沟加剧。 | 智能推荐系统、社交媒体算法、远程办公。 |
| **知识与创新** | 研究速度加快，跨学科合作更便捷；AI 可协助创作与实验设计。 | GPT‑4 论文写作、AlphaFold 蛋白质结构预测。 |
| **教育** | 个性化学习路径、可实时评估与反馈。 | AI 辅导、虚拟实验室。 |
| **健康医疗** | 诊断精准、个性化治疗、医疗资源优化。 | 病理图像识别、机器人手术。 |
| **环境与可持续** | 优化能源调度、精准农业、污染监测。 | AI 管理电网、智能灌溉系统。 |
| **安全与治理** | 提升预测与预警能力，亦带来隐私与安全风险。 | 预测恐怖袭击、自动化网络防御。 |
| **伦理与法律** | 新的责任归属、算法偏见与透明度挑战。 | AI 伦理框架、算法审计。 |

**总体趋势**  
- **加速创新与生产**：AI 可在几分钟内完成传统需要数月的分析与设计，显著提升文明发展速度。  
- **重塑人类角色**：人类从“执行者”转向“设计者”和“监督者”，但也面临职业转型和社会适应的挑战。  
- **伦理与治理新课题**：如何在高速技术迭代中确保公平、透明与安全，成为全球治理的重要议题。  

**结论**  
AI 的技术潜能与广泛应用将不可避免地改写经济、社会、文化乃至价值观的方方面面，既提供前所未有的发展机遇，也带来前所未有的风险与挑战。人类文明的未来取决于我们如何把握、引导并规范这一变革。


## 2.3 实现翻译应用 — 基于聊天模型和指令模板

In [5]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# 创建一个ChatOllama对象
chat = ChatOllama(model="gpt-oss:20b")
# 配置聊天模板信息
template = "你是一个翻译助理，请将用户输入的内容由{input_language}直接翻译为{output_language}."  # 设置翻译任务的基本指令
system_message_prompt = SystemMessagePromptTemplate.from_template(template)                     # 基于template创建系统消息指令，向聊天模型明确翻译任务要求
human_template = "{text}"                                                                       # 简单定义人类校址指令模板，仅包含待翻译文本
human_massage_prompt = HumanMessagePromptTemplate.from_template(human_template)                 # 定义人类消息指令
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_massage_prompt])   # 将系统消息指令和人类消息指令组合为完整的聊天指令，用于与聊天模型交互

# 输入信息并获取翻译结果
print(chat.invoke(chat_prompt.format_prompt(input_language="英语",
                                            output_language="中文",
                                            text="Artificial Intelligence (AI) will have a profound impact on human civilization in the coming years.").to_messages()))

content='人工智能（AI）将在未来几年对人类文明产生深远影响。' response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-12-30T09:52:47.187864775Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'done_reason': 'stop', 'total_duration': 882362707, 'load_duration': 131582167, 'prompt_eval_count': 246, 'prompt_eval_duration': 4992843, 'eval_count': 18, 'eval_duration': 73161728} id='run-ca964749-bf5b-4008-af9c-85016af2494b-0'
